In [1]:
import os
import time
import copy
import torch
import random
import numpy as np
import torch.nn as nn
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from torch.utils.data import random_split
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# ------------------------------------------------------------------------------
# 1. motion-sense Dataset
# ------------------------------------------------------------------------------
class MotionSenseDataset(Dataset):
    """
    Motion-Sense Dataset Loader
    - 입력: (T, 6) -> (userAcceleration.x, y, z, rotationRate.x, y, z)
    - 라벨: 폴더명(dws, ups, wlk, jog, sit, std)을 파싱하여 인덱스로 매핑
    """

    def __init__(
        self,
        root_dir,
        window_size=128,
        step_size=64,
        normalize=True,
        target_subjects=None,
        scaler=None
    ):
        self.root_dir = Path(root_dir)
        self.window_size = window_size
        self.step_size = step_size
        self.normalize = normalize

        # Motion-Sense의 데이터 폴더 경로 (이미지 기준 A_DeviceMotion_data 폴더)
        self.data_dir = self.root_dir / "A_DeviceMotion_data"

        # 1) 데이터 로드 및 통합
        df_all = self._load_all_data()

        if target_subjects is not None:
            df_all = df_all[df_all['subject_id'].isin(target_subjects)].copy()
            print(f"Dataset initialized with subjects: {target_subjects}")
            print(f"Total rows after filtering: {len(df_all)}")

        # 2) 라벨 -> 인덱스 매핑
        # Motion-Sense의 6개 클래스: dws, ups, wlk, jog, sit, std
        activities = ['dws', 'jog', 'sit', 'std', 'ups', 'wlk']
        self.label2idx = {label: i for i, label in enumerate(activities)}
        self.idx2label = {i: label for label, i in self.label2idx.items()}
        df_all["label_idx"] = df_all["activity"].map(self.label2idx)

        # 3) 정규화 (StandardScaler)
        # MotionSense 컬럼: userAcceleration.x/y/z (acc), rotationRate.x/y/z (gyro)
        feat_cols = [
            "userAcceleration.x", "userAcceleration.y", "userAcceleration.z",
            "rotationRate.x", "rotationRate.y", "rotationRate.z"
        ]
        feats = df_all[feat_cols].values.astype(np.float32)

        if self.normalize:
            if scaler is None:
                # 스케일러가 없으면(Train용) -> 새로 맞춤(fit)
                self.scaler = StandardScaler()
                feats = self.scaler.fit_transform(feats)
            else:
                # 스케일러가 있으면(Test용) -> 기존 것 사용(transform)
                self.scaler = scaler
                feats = self.scaler.transform(feats)
        else:
            self.scaler = None

        df_all[feat_cols] = feats

        # 4) 슬라이딩 윈도우 생성 (Subject, Activity, Trial 별로 그룹화)
        X_list = []
        y_list = []

        # trial_id는 각 csv 파일을 구분하기 위해 _load_all_data에서 생성해야 함
        for _, g in df_all.groupby(["subject_id", "activity", "trial_id"]):
            g = g.sort_values("timestamp_idx").reset_index(drop=True)

            data = g[feat_cols].values
            labels = g["label_idx"].values
            n = len(g)

            if n < window_size:
                continue

            for start in range(0, n - window_size + 1, step_size):
                end = start + window_size
                w_data = data[start:end]
                w_labels = labels[start:end]

                # 윈도우 라벨 (Mode)
                majority_label = np.bincount(w_labels).argmax()

                X_list.append(w_data.astype(np.float32))
                y_list.append(majority_label)

        self.X = np.stack(X_list) if len(X_list) > 0 else np.zeros((0, window_size, 6), dtype=np.float32)
        self.y = np.array(y_list, dtype=np.int64)

        print(f"[MotionSenseDataset] windows: {len(self.X)}, classes: {len(self.label2idx)}")
        print(f"Classes map: {self.label2idx}")

    def _load_all_data(self):
        """
        A_DeviceMotion_data 내부의 모든 폴더를 순회하며 CSV 로드
        """
        all_dfs = []

        # data_dir 내부의 폴더들 (예: dws_1, jog_9 ...)
        if not self.data_dir.exists():
             raise FileNotFoundError(f"Directory not found: {self.data_dir}")

        for folder in os.listdir(self.data_dir):
            folder_path = self.data_dir / folder
            if not folder_path.is_dir():
                continue

            # 폴더명 파싱 (예: dws_1 -> activity=dws, subject=1)
            parts = folder.split('_')
            activity_label = parts[0]
            subject_id = parts[1]

            # 폴더 내 csv 파일 읽기 (보통 sub_1.csv 같은 형태)
            for csv_file in os.listdir(folder_path):
                if not csv_file.endswith(".csv"):
                    continue

                file_path = folder_path / csv_file
                df = pd.read_csv(file_path)

                # Unnamed: 0 컬럼이 타임스탬프 역할(인덱스)
                if "Unnamed: 0" in df.columns:
                    df = df.rename(columns={"Unnamed: 0": "timestamp_idx"})
                else:
                    df["timestamp_idx"] = range(len(df))

                df["activity"] = activity_label
                df["subject_id"] = int(subject_id)
                df["trial_id"] = folder  # 폴더명 자체를 trial 식별자로 사용

                all_dfs.append(df)

        return pd.concat(all_dfs, ignore_index=True)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.tensor(self.y[idx], dtype=torch.long)


# ------------------------------------------------------------------------------
# 2. ASF Model Components
# ------------------------------------------------------------------------------

class LatentEncoder(nn.Module):
    def __init__(self, input_channels=9, latent_dim=64):
        super().__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, latent_dim, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(latent_dim)

    def forward(self, x):
        x = x.transpose(1, 2)
        h = F.relu(self.bn1(self.conv1(x)))
        h = F.relu(self.bn2(self.conv2(h)))
        s = F.relu(self.bn3(self.conv3(h)))
        s = s.transpose(1, 2)
        return s

class FlowComputer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, s):
        B, T, D = s.shape

        flow_raw = s[:, 1:, :] - s[:, :-1, :]
        flow_mag = torch.norm(flow_raw, dim=-1, keepdim=True)
        flow_dir = flow_raw / (flow_mag + 1e-8)

        flow_features = torch.cat(
            [flow_raw, flow_mag.expand(-1, -1, D), flow_dir],
            dim=-1
        )
        return flow_features, flow_raw, flow_mag

class FlowEncoder(nn.Module):
    def __init__(self, flow_dim, hidden_dim=64, num_heads=4):
        super().__init__()
        self.flow_embed = nn.Linear(flow_dim, hidden_dim)
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads,
            batch_first=True
        )
        self.flow_conv1 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.flow_conv2 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=1, padding=0)
        self.bn2 = nn.BatchNorm1d(hidden_dim)

    def forward(self, flow_features):
        h = self.flow_embed(flow_features)
        h_att, _ = self.attention(h, h, h)

        h_att = h_att.transpose(1, 2)
        h = F.relu(self.bn1(self.flow_conv1(h_att)))
        h = F.relu(self.bn2(self.flow_conv2(h)))

        h_pool = torch.mean(h, dim=-1)
        return h_pool

class StateTransitionPredictor(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )

    def forward(self, s_t):
        B, Tm1, D = s_t.shape
        inp = s_t.reshape(B * Tm1, D)
        out = self.net(inp)
        return out.reshape(B, Tm1, D)

class ASFDCLClassifier(nn.Module):
    def __init__(self,
                 input_channels=9,
                 latent_dim=64,
                 hidden_dim=64,
                 num_classes=6,
                 num_heads=4,
                 projection_dim=128):
        super().__init__()

        self.num_classes = num_classes
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim

        self.latent_encoder = LatentEncoder(input_channels, latent_dim)
        self.flow_computer = FlowComputer()
        self.flow_encoder = FlowEncoder(latent_dim * 3, hidden_dim, num_heads)
        self.state_predictor = StateTransitionPredictor(latent_dim, hidden_dim)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

        self.flow_prototypes = nn.Parameter(
            torch.randn(num_classes, hidden_dim)
        )

        self.projection_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, projection_dim)
        )

    def forward(self, x, return_details=False):
        s = self.latent_encoder(x)

        s_t = s[:, :-1, :]
        s_next = s[:, 1:, :]
        s_pred_next = self.state_predictor(s_t)

        flow_features, flow_raw, flow_mag = self.flow_computer(s)

        h = self.flow_encoder(flow_features)

        z = self.projection_head(h)
        z = F.normalize(z, dim=-1)

        logits = self.classifier(h)

        if not return_details:
            return logits

        details = {
            "s": s,
            "s_t": s_t,
            "s_next": s_next,
            "s_pred_next": s_pred_next,
            "flow_features": flow_features,
            "flow_raw": flow_raw,
            "flow_mag": flow_mag,
            "h": h,
            "z": z,
            "prototypes": self.flow_prototypes
        }
        return logits, details


# ------------------------------------------------------------------------------
# 3. Dynamics-aware Contrastive Loss
# ------------------------------------------------------------------------------
def compute_contrastive_loss(z, labels, temperature=0.07):
    B = z.shape[0]
    device = z.device

    sim_matrix = torch.mm(z, z.t()) / temperature

    labels_expanded = labels.unsqueeze(1)
    positive_mask = (labels_expanded == labels_expanded.t()).float()

    positive_mask = positive_mask - torch.eye(B, device=device)

    mask = torch.eye(B, device=device).bool()
    sim_matrix_masked = sim_matrix.masked_fill(mask, float('-inf'))

    exp_sim = torch.exp(sim_matrix_masked)

    pos_sim = (exp_sim * positive_mask).sum(dim=1)

    all_sim = exp_sim.sum(dim=1)

    has_positive = positive_mask.sum(dim=1) > 0

    if has_positive.sum() == 0:
        return torch.tensor(0.0, device=device)

    loss = -torch.log(pos_sim[has_positive] / (all_sim[has_positive] + 1e-8))

    return loss.mean()


# ------------------------------------------------------------------------------
# 4. ASF-DCL Losses: CE + L_dyn + L_flow_prior + L_proto + L_contrast
# ------------------------------------------------------------------------------
def compute_asf_dcl_losses(logits, details, labels,
                           lambda_dyn=0.1,
                           lambda_flow=0.05,
                           lambda_proto=0.1,
                           lambda_contrast=0.15,
                           dyn_classes=(0, 1, 4, 5),
                           static_classes=(2, 3),
                           dyn_target=0.7,
                           static_target=0.1,
                           proto_tau=0.1,
                           contrast_temp=0.07):
    device = logits.device

    cls_loss = F.cross_entropy(logits, labels, label_smoothing=0.05)

    s_next = details["s_next"]
    s_pred_next = details["s_pred_next"]
    dyn_loss = F.mse_loss(s_pred_next, s_next)

    flow_mag = details["flow_mag"]
    B, Tm1, _ = flow_mag.shape
    flow_mean = flow_mag.mean(dim=1).view(B)

    dyn_mask = torch.zeros_like(flow_mean, dtype=torch.bool)
    static_mask = torch.zeros_like(flow_mean, dtype=torch.bool)
    for c in dyn_classes:
        dyn_mask = dyn_mask | (labels == c)
    for c in static_classes:
        static_mask = static_mask | (labels == c)

    flow_prior_loss = torch.tensor(0.0, device=device)
    if dyn_mask.any():
        dyn_flow = flow_mean[dyn_mask]
        flow_prior_loss = flow_prior_loss + F.mse_loss(
            dyn_flow, torch.full_like(dyn_flow, dyn_target)
        )
    if static_mask.any():
        static_flow = flow_mean[static_mask]
        flow_prior_loss = flow_prior_loss + F.mse_loss(
            static_flow, torch.full_like(static_flow, static_target)
        )

    h = details["h"]
    prototypes = details["prototypes"]

    h_norm = F.normalize(h, dim=-1)
    proto_norm = F.normalize(prototypes, dim=-1)

    sim = h_norm @ proto_norm.t()
    proto_logits = sim / proto_tau
    proto_loss = F.cross_entropy(proto_logits, labels, label_smoothing=0.05)

    z = details["z"]
    contrast_loss = compute_contrastive_loss(z, labels, temperature=contrast_temp)

    total_loss = (
        cls_loss +
        lambda_dyn * dyn_loss +
        lambda_flow * flow_prior_loss +
        lambda_proto * proto_loss +
        lambda_contrast * contrast_loss
    )

    loss_dict = {
        "total": total_loss.item(),
        "cls": cls_loss.item(),
        "dyn": dyn_loss.item(),
        "flow_prior": flow_prior_loss.item(),
        "proto": proto_loss.item(),
        "contrast": contrast_loss.item()
    }
    return total_loss, loss_dict


# ------------------------------------------------------------------------------
# 5. Train / Evaluation
# ------------------------------------------------------------------------------
def train_epoch(model, dataloader, optimizer, device,
                lambda_dyn=0.1, lambda_flow=0.05,
                lambda_proto=0.1, lambda_contrast=0.15):
    model.train()
    total_loss = 0

    all_preds = []
    all_labels = []

    loss_accumulator = {
        "cls": 0.0,
        "dyn": 0.0,
        "flow_prior": 0.0,
        "proto": 0.0,
        "contrast": 0.0
    }

    for x, y in dataloader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        logits, details = model(x, return_details=True)
        loss, loss_dict = compute_asf_dcl_losses(
            logits, details, y,
            lambda_dyn=lambda_dyn,
            lambda_flow=lambda_flow,
            lambda_proto=lambda_proto,
            lambda_contrast=lambda_contrast
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        for k in loss_accumulator.keys():
            loss_accumulator[k] += loss_dict[k]

        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.detach().cpu().numpy())
        all_labels.extend(y.detach().cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')

    for k in loss_accumulator.keys():
        loss_accumulator[k] /= len(dataloader)

    return avg_loss, acc, f1, loss_accumulator


def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = F.cross_entropy(logits, y)

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.detach().cpu().numpy())
            all_labels.extend(y.detach().cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds)

    return avg_loss, acc, f1, cm

def evaluate_with_bias(model, dataloader, device, bias):
    """
    Sensor Drift (Bias) Experiment Function
    bias: The constant value 'c' to add to the input signal (x + c)
    """
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            # --- [Modification for Sensor Drift] ---
            if bias != 0:
                # Add constant 'c' to the original signal x
                x = x + bias
            # ---------------------------------------

            logits = model(x)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.detach().cpu().numpy())
            all_labels.extend(y.detach().cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro') # Using weighted as per your code

    return acc, f1

def analyze_mechanism(model, dataloader, device, bias, static_indices=[2, 3]):
    """
    실험 1 & 2를 동시에 수행하는 분석 함수
    - 실험 1: Bias 유무에 따른 정적 클래스의 Flow Magnitude 평균 변화 측정
    - 실험 2: Bias 유무에 따른 Confusion Matrix 변화 확인
    """
    model.eval()
    all_preds = []
    all_labels = []

    # Flow Magnitude를 저장할 리스트
    static_flows = []
    dynamic_flows = []

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            # [Drift Injection]
            if bias != 0:
                x = x + bias

            # 모델 추론 (Details 반환 활성화)
            logits, details = model(x, return_details=True)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

            # [Flow Magnitude 추출]
            # flow_mag shape: [Batch, Time-1, 1] -> 배치의 각 샘플별 평균 Flow 계산
            # [Batch]
            avg_flow_per_sample = details['flow_mag'].mean(dim=1).mean(dim=1).cpu().numpy()

            # 정적 클래스와 동적 클래스로 나누어 Flow 저장
            y_np = y.cpu().numpy()
            for i in range(len(y_np)):
                label = y_np[i]
                flow_val = avg_flow_per_sample[i]

                if label in static_indices:
                    static_flows.append(flow_val)
                else:
                    dynamic_flows.append(flow_val)

    # 결과 계산
    cm = confusion_matrix(all_labels, all_preds)
    acc = accuracy_score(all_labels, all_preds)

    avg_static_flow = np.mean(static_flows) if len(static_flows) > 0 else 0.0
    avg_dynamic_flow = np.mean(dynamic_flows) if len(dynamic_flows) > 0 else 0.0

    return acc, cm, avg_static_flow, avg_dynamic_flow
# ------------------------------------------------------------------------------
# 6. Main Training Loop
# ------------------------------------------------------------------------------
def main():
    SEED = 42
    set_seed(SEED)

    DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/HAR_data/motion-sense'
    BATCH_SIZE = 64
    NUM_EPOCHS = 50
    LEARNING_RATE = 0.001
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    LAMBDA_DYN = 0.05
    LAMBDA_FLOW = 0.02
    LAMBDA_PROTO = 0.05
    LAMBDA_CONTRAST = 0.2

    print("-"*80)
    print("ASF-DCL: Action State Flow with Dynamics-aware Contrastive Learning")
    print("-"*80)
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Epochs: {NUM_EPOCHS}")
    print(f"Learning rate: {LEARNING_RATE}")
    print()
    print(f"Lambda_dyn: {LAMBDA_DYN}")
    print(f"Lambda_flow:  {LAMBDA_FLOW}")
    print(f"Lambda_proto: {LAMBDA_PROTO}")
    print(f"Lambda_contrast: {LAMBDA_CONTRAST}")
    print()

    full_dataset = MotionSenseDataset(DATA_PATH, window_size=128, step_size=64, target_subjects=None, normalize=False)

    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size

    g_split = torch.Generator().manual_seed(SEED)
    train_subset, test_subset = random_split(full_dataset, [train_size, test_size], generator=g_split)
    print(f"Train size: {len(train_subset)}, Test size: {len(test_subset)}")

    train_indices = train_subset.indices
    train_data = full_dataset.X[train_indices]

    N, T, C = train_data.shape
    scaler = StandardScaler()
    scaler.fit(train_data.reshape(-1, C))

    full_N, full_T, full_C = full_dataset.X.shape
    full_scaled = scaler.transform(full_dataset.X.reshape(-1, full_C))
    full_dataset.X = full_scaled.reshape(full_N, full_T, full_C)

    g = torch.Generator()
    g.manual_seed(SEED)
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=2,
                              worker_init_fn=seed_worker,
                              generator=g)
    test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=2,
                             worker_init_fn=seed_worker,
                             generator=g)

    model = ASFDCLClassifier(
        input_channels=6,
        latent_dim=64,
        hidden_dim=64,
        num_classes=6,
        num_heads=4,
        projection_dim=128
    ).to(DEVICE)

    total_params = sum(p.numel() for p in model.parameters())
    print()
    print(f"Total parameters: {total_params:,}")

    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=LEARNING_RATE,
                                 weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=NUM_EPOCHS
    )

    best_acc = 0.0
    best_f1 = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    print("\n" + "-"*80)
    print("TRAINING")
    print("-"*80)

    for epoch in range(NUM_EPOCHS):
        start_time = time.time()

        train_loss, train_acc, train_f1, loss_dict = train_epoch(
            model, train_loader, optimizer, DEVICE,
            lambda_dyn=LAMBDA_DYN,
            lambda_flow=LAMBDA_FLOW,
            lambda_proto=LAMBDA_PROTO,
            lambda_contrast=LAMBDA_CONTRAST
        )

        test_loss, test_acc, test_f1, test_cm = evaluate(
            model, test_loader, DEVICE
        )

        scheduler.step()

        if test_f1 > best_f1:
            best_f1 = test_f1
            best_acc = test_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        epoch_time = time.time() - start_time
        best_acc = max(best_acc, test_acc)
        best_f1 = max(best_f1, test_f1)

        log_msg = (f"[{epoch+1:02d}/{NUM_EPOCHS}] "
                   f"Train Loss: {train_loss:.3f} | F1: {train_f1:.4f}  |  "
                   f"Test F1: {test_f1:.4f} (Best: {best_f1:.4f})")
        print(log_msg)

    print("\n" + "-"*80)
    print("EVALUATION...")
    print("-"*80)

    test_loss, test_acc, test_f1, test_cm = evaluate(
        model, test_loader, DEVICE
    )
    print(f"Final Result → Best Test F1: {best_f1:.4f} (Acc: {best_acc:.4f})")

    print("\n" + "="*80)
    print(" 🧪 HYPOTHESIS VERIFICATION EXPERIMENTS")
    print("="*80)

    model.load_state_dict(best_model_wts)

    # Motion-Sense Class Indices
    # 'dws':0, 'jog':1, 'sit':2, 'std':3, 'ups':4, 'wlk':5
    # Static Classes: sit(2), std(3)
    static_idxs = [2, 3]
    activities = ['dws', 'jog', 'sit', 'std', 'ups', 'wlk']

    # 1. Baseline Analysis (Bias = 0.0)
    acc_0, cm_0, static_flow_0, dynamic_flow_0 = analyze_mechanism(
        model, test_loader, DEVICE, bias=0.0, static_indices=static_idxs
    )

    # 2. Drift Analysis (Bias = 0.1) -> 문제가 발생했던 구간
    acc_bias, cm_bias, static_flow_bias, dynamic_flow_bias = analyze_mechanism(
        model, test_loader, DEVICE, bias=0.1, static_indices=static_idxs
    )

    print(f"\n[Experiment 1] Flow Magnitude Analysis (Sensitivity Check)")
    print("-" * 60)
    print(f"{'Condition':<15} | {'Avg Flow (Static)':<20} | {'Avg Flow (Dynamic)':<20}")
    print("-" * 60)
    print(f"{'Bias 0.0':<15} | {static_flow_0:.6f}{'':<12} | {dynamic_flow_0:.6f}")
    print(f"{'Bias 0.1':<15} | {static_flow_bias:.6f} (+{static_flow_bias-static_flow_0:.4f})  | {dynamic_flow_bias:.6f}")
    print("-" * 60)
    print(">> 증명 포인트: Bias 0.1일 때 Static Flow 값이 급격히 상승했는가? (YES여야 함)")

    print(f"\n[Experiment 2] Confusion Matrix Pattern (Misclassification Check)")
    print("-" * 60)

    # CM 출력을 보기 좋게 DataFrame으로 변환
    cm_df_0 = pd.DataFrame(cm_0, index=activities, columns=activities)
    cm_df_bias = pd.DataFrame(cm_bias, index=activities, columns=activities)

    print("\n< Confusion Matrix (Bias = 0.0) >")
    print(cm_df_0)
    print(f"\nAccuracy: {acc_0:.4f}")

    print("\n< Confusion Matrix (Bias = 0.1) >")
    print(cm_df_bias)
    print(f"\nAccuracy: {acc_bias:.4f}")

    print("-" * 60)
    print(">> 증명 포인트: Bias 0.1에서 'sit', 'std' 행(Row)이 'wlk', 'jog' 열(Column)로 이동했는가?")
    print(">> 즉, Static -> Dynamic 오분류가 발생했는가?")

    return model


if __name__ == "__main__":
    main()

--------------------------------------------------------------------------------
ASF-DCL: Action State Flow with Dynamics-aware Contrastive Learning
--------------------------------------------------------------------------------
Batch size: 64
Epochs: 50
Learning rate: 0.001

Lambda_dyn: 0.05
Lambda_flow:  0.02
Lambda_proto: 0.05
Lambda_contrast: 0.2

[MotionSenseDataset] windows: 22053, classes: 6
Classes map: {'dws': 0, 'jog': 1, 'sit': 2, 'std': 3, 'ups': 4, 'wlk': 5}
Train size: 17642, Test size: 4411

Total parameters: 95,462

--------------------------------------------------------------------------------
TRAINING
--------------------------------------------------------------------------------
[01/50] Train Loss: 0.854 | F1: 0.7359  |  Test F1: 0.8576 (Best: 0.8576)
[02/50] Train Loss: 0.440 | F1: 0.9445  |  Test F1: 0.8188 (Best: 0.8576)
[03/50] Train Loss: 0.376 | F1: 0.9662  |  Test F1: 0.7801 (Best: 0.8576)
[04/50] Train Loss: 0.353 | F1: 0.9751  |  Test F1: 0.9332 (Best: 0.